<a href="https://colab.research.google.com/github/Sudorenkoroma/HW_GI_ML/blob/main/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

Завантаження, обробка, та перевірка даних

In [5]:
data = pd.read_csv('/content/Housing.csv')
data.replace("yes", 1, inplace=True)
data.replace("no", 0, inplace=True)
data.replace("furnished", 1, inplace=True)
data.replace("semi-furnished", 0.5, inplace=True)
data.replace("unfurnished", 0, inplace=True)
print(data.dtypes)
print(data.isnull().sum() / len(data))
data.head()

price                 int64
area                  int64
bedrooms              int64
bathrooms             int64
stories               int64
mainroad              int64
guestroom             int64
basement              int64
hotwaterheating       int64
airconditioning       int64
parking               int64
prefarea              int64
furnishingstatus    float64
dtype: object
price               0.0
area                0.0
bedrooms            0.0
bathrooms           0.0
stories             0.0
mainroad            0.0
guestroom           0.0
basement            0.0
hotwaterheating     0.0
airconditioning     0.0
parking             0.0
prefarea            0.0
furnishingstatus    0.0
dtype: float64


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1.0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1.0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0.5
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1.0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1.0


Підготовка даних, поділ на навчальний та валідаційний набори

In [6]:
# Розділення даних на ознаки та цільову змінну
X = data.iloc[:, 1:]
y = data.iloc[:, :1]

# Розділення даних на навчальні та тестові підмножини
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних
X_train = MinMaxScaler().fit_transform(X_train)
X_test = MinMaxScaler().fit_transform(X_test)

# Додавання стовпця з одиницями для bias
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

функція гіпотези лінійної регресії у векторному вигляді

In [7]:
def linear_regression_hypothesis(X, w):
    return np.dot(X, w)

функція для обчислення функції втрат у векторному вигляді

In [8]:
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

функція градієнтного спуску

In [9]:
def gradient_descent_step(X, y, epoch=1, alpha=0.01):
    w = np.random.rand(X.shape[1], 1)
    for _ in range(epoch):
      y_pred = linear_regression_hypothesis(X, w)
      gradient = 2 / X.shape[0] * np.dot(X.T, (y_pred - y))
      w -= alpha * gradient
    return w

пошук найкращих парамемтрів W за допомогою функції градієнтного спуску

In [10]:
w_gradient = gradient_descent_step(X_train, y_train, epoch=515)

пошук цих же параметрів за допомогою аналітичного рішення

In [11]:
w_analytical = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)

використання LinearRegression з бібліотеки scikit-learn

In [12]:
model = LinearRegression()
model.fit(X_train[:, 1:], y_train)

LinearRegression()

порівняння результатів

In [14]:
mse_gradient_descent = mse_loss(y_test, linear_regression_hypothesis(X_test, w_gradient))
mse_analytical = mse_loss(y_test, linear_regression_hypothesis(X_test, w_analytical))
mse_sklearn = mean_squared_error(y_test, model.predict(X_test[:, 1:]))

print(f"MSE Gradient Descent loss: {mse_gradient_descent}, weights:{w_gradient}")
print(f"MSE Analytical Solution loss:{mse_analytical}, weights:{w_analytical}")
print(f"MSE Scikit-Learnloss loss:{mse_sklearn}, weights:{model.coef_}")

MSE Gradient Descent loss: 1903303330345.646, weights:[[1736753.01331472]
 [1026968.08338627]
 [ 977731.42908559]
 [ 949972.28762977]
 [1041405.28054661]
 [ 963245.74521108]
 [ 362689.47916381]
 [ 385740.09048558]
 [ 370753.93210635]
 [ 985142.83157622]
 [ 835096.2110354 ]
 [ 574156.8398843 ]
 [ 647092.91426236]]
MSE Analytical Solution loss:1903749221373.3342, weights:[[1843354.22905646]
 [3431599.62951728]
 [ 392872.46193802]
 [3291351.79844388]
 [1218669.49338866]
 [ 366824.19239248]
 [ 233146.76562655]
 [ 393159.77872584]
 [ 687881.31095702]
 [ 785550.57929543]
 [ 677269.54192885]
 [ 629901.66084592]
 [ 420794.23654311]]
MSE Scikit-Learnloss loss:1903749221373.3274, weights:[[3431599.62951733  392872.461938   3291351.79844388 1218669.49338865
   366824.19239248  233146.76562655  393159.77872584  687881.31095702
   785550.57929543  677269.54192884  629901.66084591  420794.23654311]]
